In [1]:
import nltk
import pandas as pd
import sklearn
import string
import re
from sklearn.datasets import fetch_20newsgroups



In [2]:
data = fetch_20newsgroups(subset='train', shuffle=True)

In [3]:
data.DESCR.split('\n')

['.. _20newsgroups_dataset:',
 '',
 'The 20 newsgroups text dataset',
 '------------------------------',
 '',
 'The 20 newsgroups dataset comprises around 18000 newsgroups posts on',
 '20 topics split in two subsets: one for training (or development)',
 'and the other one for testing (or for performance evaluation). The split',
 'between the train and test set is based upon a messages posted before',
 'and after a specific date.',
 '',
 'This module contains two loaders. The first one,',
 ':func:`sklearn.datasets.fetch_20newsgroups`,',
 'returns a list of the raw texts that can be fed to text feature',
 'extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`',
 'with custom parameters so as to extract feature vectors.',
 'The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,',
 'returns ready-to-use features, i.e., it is not necessary to use a feature',
 'extractor.',
 '',
 '**Data Set Characteristics:**',
 '',
 '    =================   ==========

In [4]:
df = pd.DataFrame({'Text': data.data, 'Label':data.target})

In [5]:
df.Text[10]

'From: irwin@cmptrc.lonestar.org (Irwin Arnstein)\nSubject: Re: Recommendation on Duc\nSummary: What\'s it worth?\nDistribution: usa\nExpires: Sat, 1 May 1993 05:00:00 GMT\nOrganization: CompuTrac Inc., Richardson TX\nKeywords: Ducati, GTS, How much? \nLines: 13\n\nI have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs\nvery well, paint is the bronze/brown/orange faded out, leaks a bit of oil\nand pops out of 1st with hard accel.  The shop will fix trans and oil \nleak.  They sold the bike to the 1 and only owner.  They want $3495, and\nI am thinking more like $3K.  Any opinions out there?  Please email me.\nThanks.  It would be a nice stable mate to the Beemer.  Then I\'ll get\na jap bike and call myself Axis Motors!\n\n-- \n-----------------------------------------------------------------------\n"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx\nirwin@cmptrc.lonestar.org    DoD #0826          (R75/6)\n-------------------------------------------

In [6]:
df.head()

,Text,Label
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [38]:
def clean_text(text):
    
    rgx = re.compile(f'[{re.escape(string.punctuation)}]' )
    text = rgx.sub('',text)
    text= text.lower()
    text = text.split('\n')
    text = ' '.join([x for x in text])

    return text

In [39]:
df['cleaned_text'] = df['Text'].apply(lambda x: clean_text(x))

In [40]:
df.head()

,Text,Label,cleaned_text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,from lerxstwamumdedu wheres my thing subject w...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,from guykuocarsonuwashingtonedu guy kuo subjec...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,from twillisececnpurdueedu thomas e willis sub...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,from jgreenamber joe green subject re weitek p...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,from jcmheadcfaharvardedu jonathan mcdowell su...


In [41]:
df.iloc[2,2]

'from twillisececnpurdueedu thomas e willis subject pb questions organization purdue university engineering computer network distribution usa lines 36  well folks my mac plus finally gave up the ghost this weekend after starting life as a 512k way back in 1985  sooo im in the market for a new machine a bit sooner than i intended to be  im looking into picking up a powerbook 160 or maybe 180 and have a bunch of questions that hopefully somebody can answer   does anybody know any dirt on when the next round of powerbook introductions are expected  id heard the 185c was supposed to make an appearence this summer but havent heard anymore on it  and since i dont have access to macleak i was wondering if anybody out there had more info   has anybody heard rumors about price drops to the powerbook line like the ones the duos just went through recently   whats the impression of the display on the 180  i could probably swing a 180 if i got the 80mb disk rather than the 120 but i dont really hav

In [ ]:
# stopwords? punctuation? abbreviations?